In [1]:
from os import listdir
from xml.etree import ElementTree
import random
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from matplotlib import pyplot as plt 
from matplotlib import image
from skimage.transform import resize
from matplotlib.patches import Rectangle
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.model import mold_image
from mrcnn.utils import Dataset
from mrcnn import visualize
import os
import pandas as pd
import numpy as np
import cv2
from shapely.geometry import Polygon

Using TensorFlow backend.


In [2]:
# define the prediction configuration
class PredictionConfig(Config):
    # define the name of the configuration
    NAME = "pictureframe_cfg"
    # number of classes (background + kangaroo)
    NUM_CLASSES = 1 + 1
    # simplify GPU config
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

In [3]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
# create config
cfg = PredictionConfig()

# define the model
model = MaskRCNN(mode='inference', model_dir='./', config=cfg)

# load model weights
model.load_weights('mask_rcnn_pictureframe_cfg_0001.h5', by_name=True)
# evaluate model on training dataset

In [ ]:
def predict(img):
    
    scaled_image = mold_image(img, cfg)
    # convert image into one sample
    sample = expand_dims(scaled_image, 0)
    yhat = model.detect(sample, verbose=0)[0]
    #print(yhat)
    ax = plt.gca()
    box = yhat['rois'][0]
    conf = yhat['scores'][0]
    # get coordinates
    y1, x1, y2, x2 = box
     
    return x1 , y1, x2, y2, conf

img = image.imread('/Users/stefgielen/Documents/school 2021-2022/thesis/Python_Workspace/CustCNN/Testfotos/kader.jpg')

scaled_image = mold_image(img, cfg)
# convert image into one sample
sample = expand_dims(scaled_image, 0)

yhat = model.detect(sample, verbose=0)[0]
#print(yhat)

plt.imshow(img)

ax = plt.gca()
i=0
for score in yhat['scores']:
    print(score)
    if (score>0):
        box = yhat['rois'][i]
        # get coordinates
        y1, x1, y2, x2 = box
        # calculate width and height of the box
        width, height = x2 - x1, y2 - y1
        # create the shape
        rect = Rectangle((x1, y1), width, height, fill=False, color='red', lw=6)
        # draw the box
        ax.add_patch(rect)
        #ax.annotate(str(score), (x1, y1-5),fontsize=20,color='red')
        ax.text(x1+3, y1-5, (('Pictureframe: ')+ (str(score*100)[:2]) +'%'), style='normal',
        bbox={'facecolor': 'red', 'pad': 10})
    i+=1
plt.rcParams["figure.figsize"] = (20,20)
plt.show()
ax=get_ax(1)
#ding=(yhat['class_ids'])
visualize.display_instances(img, yhat['rois'], yhat['masks'], yhat['class_names'], 
                            yhat['scores'], ax=ax, title="Predictions")

In [ ]:
BASE_PATH = '/Users/stefgielen/Documents/school 2021-2022/thesis/Python_Workspace/CustCNN'
DATA_PATH = os.path.sep.join([BASE_PATH, "Trimmeddata_bigger_test"])
VAL_IMAGES_PATH = os.path.sep.join([DATA_PATH, "test/images"])

VAL_IMAGES_PATH ='/Users/stefgielen/Documents/school 2021-2022/thesis/Python_Workspace/CustCNN/Trimmeddata_bigger_test/typevoorbeelden'

print(VAL_IMAGES_PATH)
VAL_ANNOTS_PATH = os.path.sep.join([DATA_PATH, "test/_annotations.csv"])
print(VAL_ANNOTS_PATH)

In [ ]:
#annotaties inladen

df=pd.read_csv(VAL_ANNOTS_PATH)
df = df.drop('class', axis=1)
df = df.drop('width', axis=1)
df = df.drop('height', axis=1)
df.head(5)

In [ ]:
def calculate_iou(box_1, box_2):
    poly_1 = Polygon(box_1)
    poly_2 = Polygon(box_2)
    iou = poly_1.intersection(poly_2).area / poly_1.union(poly_2).area
    return iou


In [ ]:
import time
from tensorflow.keras.preprocessing.image import load_img
tijden=0
i=0
ious = []
PRED_PATH = '/Users/stefgielen/Documents/school 2021-2022/thesis/Python_Workspace/Resultaten/MaskRCNN/typeset/annotations'
for filename in os.listdir(VAL_IMAGES_PATH):
    i += 1
    #print(filename)
    #print(os.path.splitext(filename)[0])
    #print("foto nr: ",i,"\n", filename)
    image_path =  os.path.sep.join([VAL_IMAGES_PATH,filename])
    #print(downloaded_image_path)
    if filename != '.DS_Store' and filename != 'd5b7a75fa40665c1_jpg.rf.f2341a5242426e1d325adf81bc2ab39f.jpg' and filename!='1c18953e11ebb4e3_jpg.rf.3c8dda944df26f79b673aed3da36ac2c.jpg':
        if filename == 'c9fcdc17b0fe09e3_jpg.rf.18b51d9004fec87a8a599bfb26e1f83e.jpg':
            print("foto nr: ",i)
            print(filename)
            img = cv2.imread(image_path)        
            originalIm = load_img(image_path)


            begin = time.time()
            (xmin, ymin, xmax, ymax,conf) = predict(img)
            print(xmin, ymin, xmax, ymax)
            end = time.time()
            #print(end-begin)
            tijden += (end - begin)

            #prediction omzetten naar correcte grootte

            """w, h = originalIm.size
            xmin = int(xmin * w)
            ymin = int(ymin * h)
            xmax = int(xmax * w)
            ymax = int(ymax * h)"""


            image = cv2.imread(image_path)
            #bounding box tekenen op foto 
            #TODO aanpassen voor meerdere per foto
            cv2.rectangle(image, (xmin, ymin), (xmax, ymax),(0, 255, 0), 2)

            row = df.loc[df['filename'] == filename]
            row = row.to_numpy()
            if row.size != 0:
                startX = (row[0][1])
                startY = (row[0][2])
                endX= (row[0][3])
                endY= (row[0][4])


            box_1 = [[xmin, ymin], [xmax, ymin], [xmax, ymax], [xmin, ymax]]
            box_2 = [[startX, startY], [endX, startY], [endX, endY], [startX, endY]]


            iou=calculate_iou(box_1, box_2)

            iou = (round(iou, 2))
            ious.append(iou)
            cv2.rectangle(image, (int(startX), int(startY)), (int(endX), int(endY)),(0, 0, 255), 2)
            cv2.rectangle(image, (xmin, ymin), (xmax, ymax),(0, 255, 0), 2)
            if iou != 0:
                cv2.rectangle(image, (xmin,ymin+20), (xmin+70, ymin), (0,255,0), -1)
                cv2.putText(img=image,text=('IOU: '+str(iou)), org=(xmin+5, ymin+15), fontFace=cv2.FONT_HERSHEY_DUPLEX , fontScale=0.4, color=(0, 0, 0),thickness=1) 

            #filename = str(i)+'.jpg'

            path ='/Users/stefgielen/Documents/school 2021-2022/thesis/Python_Workspace/Resultaten/MaskRCNN/typeset/images'
            cv2.imwrite(os.path.join(path , filename), image)




            annotation_path =  os.path.sep.join([PRED_PATH,(os.path.splitext(filename)[0]+'.txt')])
            file = open(annotation_path, 'w')
            file.write( " ".join(['Picture-frame', str(conf), str(xmin),  str(ymin), str(xmax), str(ymax)]))


tijden = tijden/i           
print(tijden)



In [ ]:
print(len(ious))
ious = np.array(ious)
plt.title("Histogram IOU's Mask-RCNN")
plt.xlabel("IOU")
plt.ylabel("foto's")
plt.hist(x=ious, bins=20, color='#000000',
                            alpha=0.7, rwidth=0.8)
plt.axvline(0.75, color='r', linestyle='dashed', linewidth=2)
plt.text(0.77, 20*0.9, 'IOU=0.75',color='r')
plt.ylim(ymin=0, ymax = 25)
plt.show() 